# SHP2CSV algorithm
## Importing libraries

In [1]:
from shapely.geometry import shape
from shapely.ops import unary_union
import fiona
import networkx as nx
import itertools
import matplotlib.pyplot as plt
from IPython.core.debugger import Tracer
from shutil import copyfile
import pandas as pd

ImportError: cannot import name 'ft2font'

## Importing and reading shp file

## Converting `geoms` into a planar graph

In [9]:
file_info = fiona.open("shp/Transmilenio/Transmilenio.shp")

## Creating a Graph object in order to print it
this part of the code is not needed, it is just for having an idea of the graph that is going to be exported

## Plotting the graph

## Creating sets

In [10]:
y = set()
x = set()
lengths = set()

nodes = set()
edges = set()
edges_prime = set()

nodes_norm = list()
edges_norm = set()

## Creating a list of nodes

In [11]:
for line in file_info:
    for node in line['geometry']['coordinates']:
        nodes.add((round(node[0],3),round(node[1],3)))
        x.add(round(node[0],3))
        y.add(round(node[1],3))

nodes = list(nodes)

## Normalizing the nodes coordinates

In [12]:
# Getting the max and min for x and y axis
x_max = max(x)
x_min = min(x)
y_max = max(y)
y_min = min(y)

In [13]:
df3 = pd.DataFrame(data=[[x_min,x_max,y_min,y_max]], columns=['x_min','x_max','y_min','y_max'])
df3.to_csv('../../distributions/boundariesTrasnmilenio.csv', index=False)

In [14]:
# Normalizing

for node in nodes:
    node_norm = (nodes.index(node), [(node[0]-x_min)/(x_max-x_min),(node[1]-y_min)/(y_max-y_min)])
    nodes_norm.append(node_norm)
    
nodes_norm = [(item[0], [round(item[1][0],3), round(item[1][1],3)]) for item in nodes_norm]

## Creating a list of edges

In [15]:
for line in file_info:
    last_coord = None
    for coord in line['geometry']['coordinates']:
        if last_coord != None:
            length = ((coord[0]-last_coord[0])**2+(coord[1]-last_coord[1])**2)**0.5
            edge = ((nodes.index((round(last_coord[0],3),round(last_coord[1],3))),nodes.index((round(coord[0],3),round(coord[1],3)))),length,line['properties']['Id'])
            edges.add(edge)
        last_coord = coord
        
    
edges = [edge for edge in edges if edge[0][1] != edge[0][0]]


## Normalizing the cost of the edges

In [16]:
length_max = max(list(zip(*edges))[1])
length_min = min(list(zip(*edges))[1])

# normalizing length
edges_norm = [(edges.index(edge),edge[0],(edge[1]-length_min)/(length_max-length_min),edge[2]) for edge in edges]

## Exporting results to csv

In [17]:
# Writing csv
import csv

# vertices
with open("vertices.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in nodes_norm:
        writer.writerow([val[0],val[1][0],val[1][1]])
        
# edges
with open("edges.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in edges_norm:
        writer.writerow([val[0], val[1][0], val[1][1], val[2], val[3]])

copyfile('edges.csv', '../../streets/edgesTransmilenio.csv');
copyfile('vertices.csv', '../../streets/verticesTransmilenio.csv');

In [18]:
coords = []

for coord in list(zip(*edges))[0]:
    coord1 = (nodes[coord[0]], nodes[coord[1]])
    coords.append(coord1)

In [19]:
coords

[((-74.172, 4.599), (-74.175, 4.631)),
 ((-74.095, 4.699), (-74.084, 4.688)),
 ((-74.149, 4.634), (-74.122, 4.631)),
 ((-74.104, 4.706), (-74.095, 4.699)),
 ((-74.053, 4.738), (-74.055, 4.725)),
 ((-74.175, 4.631), (-74.162, 4.641)),
 ((-74.122, 4.631), (-74.115, 4.631)),
 ((-74.1, 4.75), (-74.05, 4.756)),
 ((-74.113, 4.71), (-74.1, 4.75)),
 ((-74.162, 4.641), (-74.149, 4.634)),
 ((-74.078, 4.681), (-74.073, 4.676)),
 ((-74.113, 4.71), (-74.104, 4.706)),
 ((-74.08, 4.667), (-74.082, 4.65)),
 ((-74.083, 4.634), (-74.098, 4.613)),
 ((-74.115, 4.631), (-74.106, 4.627)),
 ((-74.069, 4.693), (-74.068, 4.683)),
 ((-74.082, 4.65), (-74.083, 4.634)),
 ((-74.08, 4.611), (-74.088, 4.6)),
 ((-74.075, 4.709), (-74.069, 4.693)),
 ((-74.095, 4.618), (-74.086, 4.609)),
 ((-74.057, 4.712), (-74.06, 4.696)),
 ((-74.076, 4.619), (-74.08, 4.611)),
 ((-74.077, 4.604), (-74.071, 4.605)),
 ((-74.129, 4.568), (-74.128, 4.563)),
 ((-74.063, 4.675), (-74.065, 4.662)),
 ((-74.133, 4.573), (-74.129, 4.568)),
 ((